In [6]:
! pip install simpy
! pip install rbfopt

import simpy
import numpy as np
import rbfopt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

  Obtaining dependency information for rbfopt from https://files.pythonhosted.org/packages/84/ec/059bd6e63ae4019d5adf83ceaf70d89a0f37c3efc3cd88e68e7af14c0506/rbfopt-4.2.6-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyomo from https://files.pythonhosted.org/packages/1d/38/d0bbc9f9865478da9aec17c888369c846ffc93b2ece915708194a241f3a3/Pyomo-6.6.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 41.4 MB/s eta 0:00:0000:0100:01


In [7]:
# Factory class
class Factory:
    def __init__(
            self, 
            env:simpy.Environment, 
            num_stages:int, 
            machine_times:list[float], 
            n_machines:list[int]
            ) -> None:
        self.env = env

        self.machines = [simpy.Resource(env, n_machines[i]) 
            for i in range(num_stages)]
        
        self.machine_times = machine_times
        self.n_machines = n_machines
        self.num_stages = num_stages

    def produce(self, m_id):
        rt = np.random.exponential(scale=1.0/self.machine_times[m_id])
        print("production time: ", rt, m_id)
        yield self.env.timeout(rt)


In [8]:
def manufacture(env:simpy.Environment, product_id:int, processor:Factory, data:list):

    for i in range(processor.num_stages):
        with processor.machines[i].request() as request:
            # yield can also be seen as a 'wait for' statement
            # wait for the machine to be ready
            yield request
            # wait for the factory to process the product
            yield env.process(processor.produce(i))
        print(f"product id {product_id} finished processing stage {i} at {env.now}")    
    data.append([product_id, env.now])

In [9]:
def setup(env:simpy.Environment, num_stages, num_machines, machine_times, delivery_interval:float):
    factory = Factory(env, num_stages,machine_times, num_machines)
    while True:
        # wait for the next product to be delivered
        yield env.timeout(np.random.exponential(1.0/delivery_interval))
        print(f"product {i} delivered at {env.now}")
        # if a new product is delivered, launch a new process that handles this product.
        env.process(manufacture(env, i, factory))

In [ ]:
NUM_MACHINES = [3, 5]
MACHINE_TIMES = [0.1,0.2]

DELIVERY_RATE = 1
MACHINE_COSTS = [300,200]
SIM_TIME = 120

def run(num_machines):
    data = []
    env = simpy.Environment()
    env.process(setup(env, len(num_machines), num_machines, MACHINE_TIMES, DELIVERY_RATE, data))
    env.run(until=SIM_TIME)
    return len(data)